**Yapacağınız yorumlar için şimdiden teşekkür ederim. Çalışmayı beğenirseniz oylamayı unutmayın ::>.  Sorulara elimden geldiği kadarıyla cevap vermeye çalışacağım. Çalışmada; sorulara ve geri bildirimlere göre düzeltme ve ekleme yapacağım.**

**Çalışmanın Özeti:**
* Veri Serinin Yüklenmesi
* Veri Setinin Görselleştirilmesi
* Ölçeklendirme
* En İyi Sınıflandırıcının Bulunması
* En İyi Sınıfılandırıcının Optimizasyonu

**Yapacağınız yorumlar; çalışmanın daha iyi ve anlaşılır olmasını sağlayacaktır.**

In [ ]:
# Gerekli Kütüphaneleri Yüklenmesi Yapılıyor


import numpy as np # linear algebra
import pandas as pd # veri işleme

#Görselleştirme Kütüphaneleri
import seaborn as sns
import matplotlib.pyplot as plt

#Machine Learning tools
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

import os
import warnings

# Çıktılarda karmaşıklığa sebep olduğu için uyarılırı iptal ediyoruz
warnings.filterwarnings("ignore")
print(os.listdir("../input"))

**Bağlam**
Veri seti, Portekiz "Vinho Verde" şarabının kırmızı ve beyaz çeşitleri ile ilgilidir. Daha fazla bilgi için Cortez ve diğ., (Modeling wine preferences by data mining from physicochemical properties, 2009) çalışmaya bakabilirsiniz.  Gizlilik ve lojistik konulardan dolayı, sadece fizikokimyasal (girişler) ve duyusal (çıktı) değişkenler mevcuttur (örneğin üzüm türleri, şarap markası, şarap satış fiyatı vb. Hakkında veri yoktur).

Bu veri kümeleri, sınıflandırma veya regresyon görevleri için kullanılabilir.. Sınıflar sıralıdır ve dengede değildir (örneğin, mükemmel veya kötü olanlardan çok daha normal şaraplar vardır).

**içerik**
Giriş değişkenleri (fizikokimyasal testlere dayanarak):
1 - sabit asit
2 - uçucu asitliği
3 - sitrik asit
4 - artık şeker
5 - klorürler
6 - serbest sülfür dioksit
7 - toplam sülfür dioksit
8 - yoğunluk
9 - pH
10 - sülfatlar
11 - alkol
Çıkış değişkeni (duyusal verilere göre):
12 - kalite (0 ile 10 arasında puan). Var olab kalite puanlar;3, 4, 5, 6, 7, 8.



In [ ]:
#Veri setinin yüklemesi yapılıyor
dataset=pd.read_csv("../input/winequality-red.csv")
dataset.head()

In [ ]:
#Kaç farklı kalite puanı olduğunu öğrenelim
print("Kalite puanları:",dataset['quality'].unique())
dataset.info()

Veri setinde toplam 1599 örnek yer almaktadır
Veri setidende eksik bilgi yoktur.
Veri setideki tüm özellikler sayısayl değer içermektedir
Şarapların için beş ayrı kalite seviyesi belirlenmiştir; 3, 4, 5, 6, 7, 8. En düşük değer 3 ve en yüksek değer 8'dir. 

In [ ]:
#Farklı kalite puanlarının görselleştirilmesi
sns.countplot(dataset['quality'])

Veri setindeki örneklerin sınıf dağılımı dengesizdir. Dengesiz sınıf dağılımı, makine öğrenmesi modelleri için istenmyen bir durumdur.

In [ ]:
#Özelliklerin kalite puanları ile ilişkisini göstermek için kullanılacak
#çizim türleri
def draw_multivarient_plot(dataset, rows, cols, plot_type):
    column_names=dataset.columns.values
    number_of_column=len(column_names)
    fig, axarr=plt.subplots(rows,cols, figsize=(22,16))

    counter=0
    for i in range(rows):
        for j in range(cols):
            if 'violin' in plot_type:
                sns.violinplot(x='quality', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'box'in plot_type :
                sns.boxplot(x='quality', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'point' in plot_type:
                sns.pointplot(x='quality',y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'bar' in plot_type:
                sns.barplot(x='quality',y=column_names[counter],data=dataset, ax=axarr[i][j])
                
            counter+=1
            if counter==(number_of_column-1,):
                break

Veri setindeki özelliklerin kalite ile ilişkilerini yorumlamak için kullanılabilecek grafik türleri şöyledir:
Keman Grafiği (Violin Plot)
Kuru Grafiği (Box Plot)
Nokta Grafiği (Point Plot)
Bar Grafiği (Bar Plot)

In [ ]:
draw_multivarient_plot(dataset,4,3,"box")

Yukarıda gösterilen dağıtımların merkezi, boxplottaki "kutu" dur. Kutunun üst kısmı 75. persentil, alt ise 25 persentildir. Diğer bir deyişle, verilerin yarısı kutu içinde dağıtılır! Ortadaki yeşil çizgi ortancadır.

Parselin diğer kısmı, "bıyık", dağıtım merkezinin ötesindeki noktaların kapsamını gösterir. Bunun ötesinde bireysel daireler aykırı değerlerdir.


Boxlot'lar, birçok veri kümesinin şeklini özetlemek için harikadır. Ayrıca, sayı bakımından bir sınırı da yoktur: sayfa üzerinde rahatça sıkışma yaptığınız için alana çok sayıda kutu yerleştirebilirsiniz.

Bununla birlikte, sadece çok sayıda olası değişken ile aralık değişkenleri ve nominal değişkenler için çalışırlar; Verilerinizin normal olarak dağılmış olduğunu varsayarlar (aksi takdirde tasarımları çok anlamlı değildir); ve bireysel değerler hakkında herhangi bir bilgi taşımazlar, sadece dağıtımı bir bütün olarak ele alırlar.

Kaynak:https://www.kaggle.com/residentmario/plotting-with-seaborn

In [ ]:
draw_multivarient_plot(dataset,4,3,"violin")

Bir keman plotu, boxplot'taki kutuyu, veriler için bir çekirdek yoğunluk tahmini ile akıllıca değiştirir. Temelde aynı verileri gösterir, ancak yanlış yorumlama yapmayı engeller. Ayrıca, boxplot'tan daha zariftir.

Kaynak:https://www.kaggle.com/residentmario/plotting-with-seaborn

In [ ]:
draw_multivarient_plot(dataset,4,3,"pointplot")

Bir nokta grafiği(point plot), dağılım diyagramı noktalarının konumuyla sayısal bir değişken için merkezi eğilimin bir tahminini temsil eder ve hata çubukları kullanarak bu tahminin etrafındaki belirsizliğin bir miktarını gösterir.

Nokta grafikleri, bir veya daha fazla kategorik değişkenin farklı seviyeleri arasındaki karşılaştırmaları odaklamak için çubuk çizimlerden(bar plor) daha kullanışlı olabilir. Bunlar etkileşimleri göstermede özellikle ustadır: Bir kategorik değişkenin seviyeleri arasındaki ilişkinin ikinci kategorik bir değişkenin seviyeleri arasında nasıl değiştiği. Her bir noktayı aynı renk tonundan birleştiren çizgiler, farklı nokta veya çubuk gruplarının yüksekliklerini karşılaştırmaktan ziyade, gözler için daha kolay olan eğimdeki farklılıklar tarafından değerlendirilmesine izin verir.

Bir nokta çiziminin sadece ortalama (ya da diğer tahmin edici) değerini gösterdiğini, ancak çoğu durumda kategorik değişkenlerin her düzeyindeki değerlerin dağılımını göstermek için daha bilgilendirici olabileceğini akılda tutmak önemlidir. Bu durumda, bir kutu veya keman grafikleri  gibi diğer yaklaşımlar daha uygun olabilir.

Kaynak:https://seaborn.pydata.org/generated/seaborn.pointplot.html

In [ ]:
draw_multivarient_plot(dataset,4,3,"bar")


**Barplot**

Dikdörtgen çubuk grafiği, nokta tahminleri ve güven aralıklarını gösteren bir grafik türüdür.

Bir çubuk grafiği, her bir dikdörtgenin yüksekliği ile sayısal bir değişken için bir merkezi eğilim tahminini temsil eder ve hata çubuklarını kullanarak bu tahminin etrafındaki belirsizliğin bir miktarını gösterir. Çubuk parseller kantitatif eksen aralığında 0 içerir ve niceliksel değişken için 0 anlamlı bir değer olduğunda iyi bir seçimdir ve ona karşı karşılaştırmalar yapmak istersiniz.

0'ın anlamlı bir değer olmadığı veri kümeleri için, bir nokta grafiği bir veya daha fazla kategorik değişkenin seviyeleri arasındaki farklara odaklanmanıza izin verecektir.

Bir çubuk çiziminin sadece ortalama (veya başka bir tahmin edici) değeri olduğunu akılda tutmak önemlidir, ancak çoğu durumda kategorik değişkenlerin her seviyesinde değer dağılımını göstermek daha bilgilendirici olabilir. Bu durumda, bir kutu veya keman grafikleri  gibi diğer yaklaşımlar daha uygun olabilir.

Kaynak: https://seaborn.pydata.org/generated/seaborn.barplot.html#seaborn.barplot

In [ ]:

def get_models():
    models=[]
    models.append(("LR",LogisticRegression()))
    models.append(("NB",GaussianNB()))
    models.append(("KNN",KNeighborsClassifier()))
    models.append(("DT",DecisionTreeClassifier()))
    models.append(("SVM rbf",SVC()))
    models.append(("SVM linear",SVC(kernel='linear')))
    
    return models

def cross_validation_scores_for_various_ml_models(X_cv, y_cv):
    print("Çapraz Doğrulama Başarı Oranları".upper())
    models=get_models()


    results=[]
    names= []

    for name, model in models:
        kfold=KFold(n_splits=10,random_state=22)
        cv_result=cross_val_score(model,X_cv, y_cv, cv=kfold,scoring="accuracy")
        names.append(name)
        results.append(cv_result)
        print("{} modelinin çapraz doğrulaması yapıldı, başarı oranı:{:0.2f}".format(name, cv_result.mean()))

   

In [ ]:
dataset_temp=dataset.copy(deep=True)
X=dataset.drop('quality', axis=1)
y=dataset['quality']

X=StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)


cross_validation_scores_for_various_ml_models(X, y)


In [ ]:
def SVM_GridSearch(X_train, X_test, y_train, y_test):
    best_score=0
    gammas=[0.001, 0.01, 0.1, 1, 10, 100]
    Cs=[0.001, 0.01, 0.1, 1, 10, 100]
    
    for gamma in gammas:
        for C in Cs:
            svm=SVC(kernel='rbf',gamma=gamma, C=C)
            svm.fit(X_train, y_train)
            
            
            score=svm.score(X_test, y_test)
            
            if score>best_score:
                y_pred=svm.predict(X_test)
                best_score=score
                best_params={'C':100, 'gamma':gamma}
        
    print("best score:",best_score)
    print("best params:",best_params)
    print("classification reports:\n",classification_report(y_test, y_pred))

In [ ]:
SVM_GridSearch(X_train, X_test, y_train, y_test)

In [ ]:



dataset_temp.loc[(dataset_temp['quality']==3),'quality']=1
dataset_temp.loc[(dataset_temp['quality']==4),'quality']=1

dataset_temp.loc[(dataset_temp['quality']==5),'quality']=2
dataset_temp.loc[(dataset_temp['quality']==6),'quality']=2

dataset_temp.loc[(dataset_temp['quality']==7),'quality']=3
dataset_temp.loc[(dataset_temp['quality']==8),'quality']=3

draw_multivarient_plot(dataset_temp,4,3,"violin")


In [ ]:
draw_multivarient_plot(dataset_temp,4,3,"box")

In [ ]:
X_temp=dataset_temp.drop('quality', axis=1)
y_temp=dataset_temp['quality']
X=StandardScaler().fit_transform(X)

X_train_temp, X_test_temp, y_train_temp, y_test_temp=train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)


cross_validation_scores_for_various_ml_models(X_temp, y_temp)

In [ ]:
SVM_GridSearch(X_train_temp, X_test_temp, y_train_temp, y_test_temp)